# Các thư viện

Ta cần cài đặt thư viện:
`pip install pycountry-convert`

In [1]:
import pandas as pd
import numpy as np

import pycountry_convert

# Đọc dữ liệu

In [2]:
weather_df = pd.read_csv('data/raw/daily_weather_data.csv')
weather_df.head(5)

,date,country,city,Latitude,Longitude,tavg,tmin,tmax,wdir,wspd,pres
0,21-07-2018,Abkhazia,Sukhumi,43.001525,41.023415,23.4,20.9,25.5,329.0,9.3,1009.6
1,22-07-2018,Abkhazia,Sukhumi,43.001525,41.023415,23.5,21.0,25.7,337.0,9.4,1010.0
2,23-07-2018,Abkhazia,Sukhumi,43.001525,41.023415,23.5,21.1,25.5,41.0,8.2,1007.7
3,24-07-2018,Abkhazia,Sukhumi,43.001525,41.023415,24.3,20.8,27.1,10.0,9.3,1004.4
4,25-07-2018,Abkhazia,Sukhumi,43.001525,41.023415,26.5,22.7,30.0,9.0,9.7,1002.0


# Thông tin về dataset

In [3]:
num_rows, num_cols = weather_df.shape
print('Số dòng: {}\nSố cột: {}'.format(num_rows, num_cols))

Số dòng: 324647
Số cột: 11


# Chuyển cột __date__ sang kiểu _datetime_

### Kiểu ban đầu

In [4]:
weather_df["date"].dtypes

dtype('O')

### Chuyển sang _datetime_

In [5]:
weather_df['date']= pd.to_datetime(weather_df['date'], format='%d-%m-%Y')
weather_df['date'].dtypes # '<M8[ns]' is datetime64[ns]

dtype('<M8[ns]')

# Thời gian thu thập dữ liệu

In [21]:
print(weather_df['date'].min())
print(weather_df['date'].max())

2018-01-01 00:00:00
2022-10-11 00:00:00


# Loại bỏ các dòng chứa các giá trị thiếu

Có 5 cột chứa giá trị rỗng: <span style="color: yellow;"> wdir, pres, wspd, tmax, tmin, tavg</span> (ta sẽ lưu tạm vào biến __numeric_cols__). Các cột còn lại không mô tả thông số về thời tiết nên tạm thời chưa nhắc đến.
Ta sẽ xử lý như sau:
- Xóa các dòng thiếu tất cả các giá trị của các cột trên

In [6]:
numeric_cols = ['tavg','tmin','tmax','wdir','wspd','pres']

### Số dòng chứa giá trị rỗng bất kì

In [7]:
any_nan = len(weather_df[weather_df.isnull().any(axis=1)])
print(f"Phần trăm các-dòng-thiếu/tổng-số-dòng: {round(any_nan * 100 / num_rows, 2)} %")

Phần trăm các-dòng-thiếu/tổng-số-dòng: 14.51 %


Số dòng thiếu khá lớn, không thể xóa hết đi được

### Các dòng thiếu tất cả giá trị quan trọng

In [8]:
weather_df[weather_df[numeric_cols].isnull().all(axis=1)]

,date,country,city,Latitude,Longitude,tavg,tmin,tmax,wdir,wspd,pres
1413,2022-06-03,Abkhazia,Sukhumi,43.001525,41.023415,NaN,NaN,NaN,NaN,NaN,NaN
1414,2022-06-04,Abkhazia,Sukhumi,43.001525,41.023415,NaN,NaN,NaN,NaN,NaN,NaN
1415,2022-06-05,Abkhazia,Sukhumi,43.001525,41.023415,NaN,NaN,NaN,NaN,NaN,NaN
1416,2022-06-06,Abkhazia,Sukhumi,43.001525,41.023415,NaN,NaN,NaN,NaN,NaN,NaN
1417,2022-06-07,Abkhazia,Sukhumi,43.001525,41.023415,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
323970,2020-12-04,Western Sahara,El Aaiún,27.125287,-13.162500,NaN,NaN,NaN,NaN,NaN,NaN
323984,2020-12-18,Western Sahara,El Aaiún,27.125287,-13.162500,NaN,NaN,NaN,NaN,NaN,NaN
323997,2020-12-31,Western Sahara,El Aaiún,27.125287,-13.162500,NaN,NaN,NaN,NaN,NaN,NaN
324133,2021-05-16,Western Sahara,El Aaiún,27.125287,-13.162500,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
all_nan = len(weather_df[weather_df[numeric_cols].isnull().all(axis=1)])
print(f"Phần trăm các-dòng-thiếu/tổng-số-dòng: {round(all_nan * 100 / num_rows, 2)} %")

Phần trăm các-dòng-thiếu/tổng-số-dòng: 2.29 %


### Phần trăm khá nhỏ, ta có thể xóa đi những dòng này:

In [10]:
weather_df.dropna(subset=numeric_cols, how='all', axis='index', inplace=True)
weather_df.shape

(317211, 11)

# Thêm cột __continent__ dựa vào cột country

Ta sẽ sử dụng thư viện <span style='color:pink;'>pycountry_convert</span>
- Thư viện này dựa vào thông tin về mã quốc gia trên wiki, [hướng dẫn sử dụng](https://pypi.org/project/pycountry-convert/)
- Ta cần đầu vào là tên quốc gia, VD: France.
- Sau đó method `country_name_to_country_alpha2` sẽ chuyển thành mã quốc gia tương ứng FR.
- Method `country_alpha2_to_continent_code` sẽ chuyển FR thành mã lục địa: EU
- Ta cần chuyển sang tên lục địa: EU -> Europe

- Nhược điểm của thư viện __pycountry-convert__: 
  - Một số quốc gia không trùng tên sẽ báo lỗi (ta sẽ lưu những quốc gia này lại và xử lý thủ công)
  - Tuy thông tin tìm được trên Wiki hiển thị kết quả về mã quốc gia, nhưng thư viện này lại báo lỗi không tìm thấy (có lẽ thư viện hơi lỗi thời)
  - Chỉ dịch đến mã lục địa, còn tên lục địa thì phải xử lý thủ công

In [11]:
country_not_found = []

def convert_countryName_to_continentCode(row):
    '''
    input: single row of a dataframe
    output: country code (AS: Asia, EU: Europe, ...)
    convert country name to continent code. If the country is not found, return country name
    '''
    try:
        # country name -> country code (France -> FR)
        cn_code = pycountry_convert.country_name_to_country_alpha2(row.country, cn_name_format='default')
        # country code -> continent code (FR -> EU)
        conti_code = pycountry_convert.country_alpha2_to_continent_code(cn_code)
    except KeyError:
        country_not_found.append(row.country)
        return row.country

    return conti_code

Cách xử lý khi báo lỗi không có quốc gia tương ứng:
- Lưu tạm kết quả là __tên quốc gia__ đó (dòng `return row.country`)
- Lưu __tên quốc gia__ không xử lý được. Tạo dict và gán thủ công (tra thông tin trên mạng)

In [12]:
weather_df['continent'] = weather_df.apply(convert_countryName_to_continentCode, axis=1)
weather_df

,date,country,city,Latitude,Longitude,tavg,tmin,tmax,wdir,wspd,pres,continent
0,2018-07-21,Abkhazia,Sukhumi,43.001525,41.023415,23.4,20.9,25.5,329.0,9.3,1009.6,Abkhazia
1,2018-07-22,Abkhazia,Sukhumi,43.001525,41.023415,23.5,21.0,25.7,337.0,9.4,1010.0,Abkhazia
2,2018-07-23,Abkhazia,Sukhumi,43.001525,41.023415,23.5,21.1,25.5,41.0,8.2,1007.7,Abkhazia
3,2018-07-24,Abkhazia,Sukhumi,43.001525,41.023415,24.3,20.8,27.1,10.0,9.3,1004.4,Abkhazia
4,2018-07-25,Abkhazia,Sukhumi,43.001525,41.023415,26.5,22.7,30.0,9.0,9.7,1002.0,Abkhazia
...,...,...,...,...,...,...,...,...,...,...,...,...
324642,2022-10-07,Western Sahara,El Aaiún,27.125287,-13.162500,22.1,18.6,25.6,3.0,24.7,1013.7,Western Sahara
324643,2022-10-08,Western Sahara,El Aaiún,27.125287,-13.162500,22.1,19.6,25.6,2.0,20.5,1013.0,Western Sahara
324644,2022-10-09,Western Sahara,El Aaiún,27.125287,-13.162500,23.4,19.6,29.6,34.0,24.0,1015.4,Western Sahara
324645,2022-10-10,Western Sahara,El Aaiún,27.125287,-13.162500,23.1,18.6,29.7,39.0,28.4,1017.6,Western Sahara


Xem kết quả một cách chi tiết hơn

In [13]:
weather_df['continent'].value_counts()

EU                                   75268
AS                                   60253
AF                                   55777
NA                                   49867
OC                                   35780
SA                                   14540
Vatican City                          1745
Congo (DRC)                           1745
Congo (Republic)                      1745
U.S. Virgin Islands                   1744
Macedonia (FYROM)                     1743
Aland Islands                         1739
French Southern Territories           1738
Netherlands Antilles                  1718
St. Barthélemy                        1700
Abkhazia                              1532
Timor-Leste                           1421
AN                                    1416
Pitcairn Islands                      1399
Western Sahara                        1396
Tristan da Cunha                       924
Transnistria                           729
Myanmar (Burma)                        647
Falkland Is

In [14]:
country_not_found = np.unique(country_not_found) 
country_not_found

array(['Abkhazia', 'Aland Islands', 'Congo (DRC)', 'Congo (Republic)',
       'Falkland Islands (Islas Malvinas)', 'French Southern Territories',
       'Macedonia (FYROM)', 'Myanmar (Burma)', 'Netherlands Antilles',
       'Pitcairn Islands', 'St. Barthélemy', 'Timor-Leste',
       'Transnistria', 'Tristan da Cunha', 'U.S. Virgin Islands',
       'Vatican City', 'Western Sahara'], dtype='<U33')

### Thay thế các quốc gia không xử lý được này

In [15]:
country_to_continentName = { 
        'Abkhazia': 'AS', 'Aland Islands': 'EU', 'Congo (DRC)': 'AF', 
        'Congo (Republic)': 'AF', 'Falkland Islands (Islas Malvinas)': 'SA', 
        'French Southern Territories': 'AN', 'Macedonia (FYROM)': 'EU', 'Myanmar (Burma)': 'AS', 
        'Netherlands Antilles': 'SA', 'Pitcairn Islands': 'OC', 'St. Barthélemy': 'NA', 
        'Timor-Leste': 'AS', 'Transnistria': 'EU', 'Tristan da Cunha': 'AF',
        'U.S. Virgin Islands': 'SA', 'Vatican City': 'EU', 'Western Sahara': 'AF'
}

In [16]:
weather_df = weather_df.replace({'continent': country_to_continentName})

In [17]:
weather_df['continent'].value_counts()

EU    81224
AS    63853
AF    61587
NA    51567
OC    37179
SA    18647
AN     3154
Name: continent, dtype: int64

### Chuyển _continent code_ này về _continent name_

In [18]:
continents_names = {"EU": "Europe", 
                    "AS": "Asia", 
                    "AF":"Africa", 
                    "NA": "North America", 
                    "OC": "Oceania",
                    "SA": "South America",
                    "AN": "Antarctica"}

In [19]:
weather_df['continent'] = weather_df['continent'].map(continents_names)
weather_df['continent'].value_counts()

Europe           81224
Asia             63853
Africa           61587
North America    51567
Oceania          37179
South America    18647
Antarctica        3154
Name: continent, dtype: int64

# Lưu dataset 

Lưu tại <span style='color:red;'>./data/cleaned/daily_weather_data.csv</span>

In [20]:
weather_df.to_csv('./data/cleaned/daily_weather_data.csv', index=False, encoding='utf-8')